In [5]:
import requests
from bs4 import BeautifulSoup
import time
import csv
from urllib.parse import urljoin
import random
import traceback
import re

# --- FUNGSI BARU UNTUK MENGAMBIL KONTEN ARTIKEL (DIPERBAIKI SESUAI INFO ANDA) ---
def get_article_content(article_url, headers):
    """
    Mengunjungi URL artikel dan mengambil teks dari <div class="media_desc">.
    Jika tidak ditemukan, akan mencoba fallback ke metode pencarian paragraf umum.
    """
    print(f"    Mengambil konten dari: {article_url}")
    article_text = ""
    try:
        # Jeda kecil SEBELUM request ke halaman artikel individu
        time.sleep(random.uniform(0.5, 1.5)) # Mengurangi sedikit jeda untuk testing

        article_response = requests.get(article_url, headers=headers, timeout=12) # Timeout sedikit diperpanjang
        article_response.raise_for_status()
        detail_article_soup = BeautifulSoup(article_response.content, 'html.parser')

        # PRIORITAS: Cari <div class="media_desc"> seperti yang Anda sebutkan
        media_desc_div = detail_article_soup.find('div', class_='media_desc')

        if media_desc_div:
            # Ambil semua teks dari dalam div ini.
            # get_text(separator='\n', strip=True) akan menggabungkan teks dari semua elemen anak,
            # memisahkan blok teks yang berbeda dengan baris baru, dan menghapus spasi ekstra.
            article_text = media_desc_div.get_text(separator='\n', strip=True)
            if article_text:
                print(f"      Konten ditemukan dalam 'div.media_desc'. Panjang: {len(article_text)} chars.")
            else:
                print(f"      'div.media_desc' ditemukan, tetapi tidak ada teks di dalamnya di {article_url}.")
        else:
            # FALLBACK jika 'div.media_desc' tidak ditemukan.
            # Anda bisa menghapus bagian fallback ini jika yakin 'media_desc' selalu ada
            # dan berisi konten yang diinginkan.
            print(f"      Peringatan: 'div.media_desc' TIDAK ditemukan di {article_url}. Mencoba fallback (area konten umum + <p>).")

            # Coba cari area konten yang lebih umum seperti sebelumnya
            # (Anda mungkin perlu menyesuaikan class_recompile ini berdasarkan inspeksi lebih lanjut jika fallback ini sering terpakai)
            content_area = detail_article_soup.find('div', class_=re.compile(r'detail__body-text|article__body|entry-content|post-content|read__content', re.IGNORECASE))

            if content_area:
                paragraphs = content_area.find_all('p', recursive=False) # Ambil <p> yang merupakan anak langsung dari content_area
                if not paragraphs: # Jika tidak ada <p> anak langsung, coba cari semua <p> di dalamnya
                    paragraphs = content_area.find_all('p')

                collected_texts = []
                for p_index, p in enumerate(paragraphs):
                    # Logika filter paragraf (bisa disesuaikan atau dihapus jika tidak perlu)
                    parent_classes_to_skip = ['lihat-juga', 'related-articles', 'baca-juga', 'artikel-terkait', 'recommendation-box', 'bottom-of-article']
                    skip_paragraph = False
                    # Cek apakah paragraf atau parentnya memiliki class yang ingin di-skip
                    current_element = p
                    while current_element and current_element != content_area and current_element != detail_article_soup.body:
                        if current_element.has_attr('class') and any(cls in current_element['class'] for cls in parent_classes_to_skip):
                            skip_paragraph = True
                            # print(f"        Skipping paragraph {p_index+1} due to parent class: {current_element['class']}")
                            break
                        current_element = current_element.parent

                    if 'script' in str(p).lower() or 'style' in str(p).lower(): # Hindari script/style di dalam p
                        skip_paragraph = True
                        # print(f"        Skipping paragraph {p_index+1} due to script/style content")

                    if not skip_paragraph:
                        text_content = p.get_text(strip=True)
                        if text_content: # Hanya tambahkan jika ada teks
                            collected_texts.append(text_content)

                article_text = "\n".join(collected_texts)
                if article_text:
                     print(f"      Konten ditemukan via FALLBACK (area konten umum + <p>). Panjang: {len(article_text)} chars.")

            if not article_text: # Jika fallback juga tidak menghasilkan apa-apa
                 print(f"      Peringatan: Tidak ada teks konten yang terambil dari {article_url} dengan metode fallback saat ini.")

    except requests.exceptions.RequestException as e:
        print(f"    Error saat mengambil konten artikel {article_url}: {e}")
    except Exception as e_detail:
        print(f"    Error saat parsing konten artikel {article_url}: {e_detail}")
        # traceback.print_exc() # Uncomment untuk debug error parsing yang lebih detail

    return article_text
# --- AKHIR FUNGSI ---

# URL awal / halaman pertama (dari contoh Anda)
current_page_url = 'https://www.detik.com/search/searchall?query=blt&page=1&result_type=relevansi'

# Untuk menyimpan semua data yang di-scrape
semua_data_scraped = []

# Opsional: Batas maksimal data yang ingin diambil
MAKSIMAL_DATA_TOTAL = 100 # DIKECILKAN LAGI untuk testing cepat pengambilan konten
jumlah_data_terscrape_total = 0

# Opsional: Batas maksimal halaman yang ingin di-scrape
MAKSIMAL_HALAMAN = None # Batasi hanya beberapa halaman untuk testing konten
jumlah_halaman_terscrape = 0

# Header, termasuk User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

print("Memulai proses scraping...")

while current_page_url and \
      (MAKSIMAL_HALAMAN is None or jumlah_halaman_terscrape < MAKSIMAL_HALAMAN) and \
      (MAKSIMAL_DATA_TOTAL is None or jumlah_data_terscrape_total < MAKSIMAL_DATA_TOTAL):

    print(f"Mengambil data dari halaman pencarian: {current_page_url}")
    jumlah_halaman_terscrape += 1

    try:
        response = requests.get(current_page_url, headers=headers, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        items_di_halaman_ini = soup.find_all('article') # Asumsi selector ini sudah baik untuk Anda

        if not items_di_halaman_ini and jumlah_halaman_terscrape > 1:
            print("Tidak ada item berita ditemukan di halaman pencarian ini, mungkin halaman terakhir atau selector item salah.")
            break
        elif not items_di_halaman_ini and jumlah_halaman_terscrape == 1:
            print("Tidak ada item berita ditemukan di halaman pencarian PERTAMA. Periksa selector item ('article') Anda!")
            break

        print(f"Ditemukan {len(items_di_halaman_ini)} item berita di halaman pencarian ini.")

        for item_index, item in enumerate(items_di_halaman_ini):
            if MAKSIMAL_DATA_TOTAL is not None and jumlah_data_terscrape_total >= MAKSIMAL_DATA_TOTAL:
                print(f"Batas maksimal {MAKSIMAL_DATA_TOTAL} data (artikel) telah tercapai.")
                current_page_url = None # Set None untuk menghentikan loop while luar
                break # Keluar dari loop for item

            try:
                title_element = item.find('h3', class_='media__title')
                judul_berita = title_element.get_text(strip=True) if title_element else "Judul tidak ditemukan"

                link_element = item.find('a')
                link_berita = link_element['href'] if link_element and link_element.has_attr('href') else "Link tidak ditemukan"

                if link_berita != "Link tidak ditemukan" and not link_berita.startswith('http'):
                    link_berita = urljoin(response.url, link_berita)

                konten_artikel = ""
                if link_berita != "Link tidak ditemukan":
                    # Memanggil fungsi untuk mengambil konten artikel
                    konten_artikel = get_article_content(link_berita, headers)
                else:
                    print(f"  Skipping pengambilan konten untuk item ke-{item_index+1} karena link tidak ditemukan.")


                data_item = {
                    'judul': judul_berita,
                    'link': link_berita,
                    'konten': konten_artikel,
                    'halaman_sumber_pencarian': current_page_url
                }
                semua_data_scraped.append(data_item)
                jumlah_data_terscrape_total += 1
                print(f"  -> Data artikel ke-{jumlah_data_terscrape_total} BERHASIL diproses: {judul_berita} (Panjang Konten: {len(konten_artikel)} chars)")

            except Exception as e_item:
                print(f"  Error saat memproses satu item berita ke-{item_index+1}: {e_item}")
                traceback.print_exc()

        if current_page_url is None: # Jika sudah di-set None oleh inner loop (batas data tercapai)
            break # Keluar dari loop while

        # --- BAGIAN UNTUK MENEMUKAN HALAMAN PENCARIAN BERIKUTNYA ---
        next_button = None
        # (Logika pencarian next_button Anda yang sudah bekerja sebelumnya bisa diletakkan di sini)
        # Contoh sederhana (pastikan ini bekerja atau gunakan logika multi-upaya Anda sebelumnya):
        if not next_button:
            next_button = soup.find('a', rel='next')
            if next_button: print("Tombol 'Next' (pencarian) ditemukan via rel='next'")
        if not next_button:
            possible_buttons = soup.find_all('a', class_=re.compile(r'pagination|page-link|next', re.IGNORECASE))
            if not possible_buttons: possible_buttons = soup.find_all('a')
            for button in possible_buttons:
                button_text = button.get_text(strip=True).lower()
                if button_text == "berikutnya" or button_text == "next" or button_text == "selanjutnya" or button_text == "»":
                    next_button = button
                    print(f"Tombol 'Next' (pencarian) ditemukan via teks persis: '{button_text}'")
                    break
            if not next_button: # Coba yang mengandung teks
                 for button in possible_buttons:
                    button_text = button.get_text(strip=True).lower()
                    if "berikutnya" in button_text or "next" in button_text or "selanjutnya" in button_text:
                        next_button = button
                        print(f"Tombol 'Next' (pencarian) ditemukan via teks mengandung: '{button_text}'")
                        break
        if not next_button:
            next_button = soup.select_one('a[title*="Berikutnya"], a[title*="Next"], a[aria-label*="Berikutnya"], a[aria-label*="Next"]')
            if next_button: print("Tombol 'Next' (pencarian) ditemukan via atribut title atau aria-label")

        if next_button and next_button.has_attr('href'):
            next_page_link = next_button['href']
            current_page_url = urljoin(response.url, next_page_link)
            print(f"Link halaman pencarian berikutnya ditemukan: {current_page_url}")
        else:
            print("Tombol 'Next' (halaman pencarian berikutnya) TIDAK ditemukan. Mengakhiri pagination.")
            current_page_url = None
        # --- AKHIR BAGIAN MENEMUKAN HALAMAN PENCARIAN BERIKUTNYA ---

        if current_page_url:
            jeda_halaman_pencarian = random.uniform(2, 5) # Jeda antar halaman pencarian
            print(f"Menunggu {jeda_halaman_pencarian:.2f} detik sebelum ke halaman pencarian berikutnya...")
            time.sleep(jeda_halaman_pencarian)

    except requests.exceptions.HTTPError as e:
        print(f"Error HTTP saat mengakses halaman pencarian {current_page_url if isinstance(current_page_url, str) else 'URL tidak valid'}: {e}")
        if hasattr(e, 'response') and e.response is not None and e.response.status_code == 404:
            print("Halaman pencarian tidak ditemukan (404).")
        break
    except requests.exceptions.RequestException as e:
        print(f"Error koneksi saat mengakses halaman pencarian {current_page_url if isinstance(current_page_url, str) else 'URL tidak valid'}: {e}")
        break
    except Exception as e:
        print(f"Terjadi kesalahan umum pada loop utama: {e}")
        traceback.print_exc()
        break

print(f"\nProses scraping selesai.")
print(f"Total data artikel yang berhasil di-scrape: {len(semua_data_scraped)} item (dari {jumlah_halaman_terscrape} halaman pencarian).")

if semua_data_scraped:
    nama_file_csv = 'data_detik_blt_dengan_konten_v2.csv'
    if semua_data_scraped:
        fieldnames = semua_data_scraped[0].keys()
        try:
            with open(nama_file_csv, mode='w', newline='', encoding='utf-8') as file_csv:
                writer = csv.DictWriter(file_csv, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(semua_data_scraped)
            print(f"Data telah disimpan ke {nama_file_csv}")
        except IOError:
            print(f"Error: Tidak bisa menulis ke file {nama_file_csv}")
        except Exception as e:
            print(f"Error saat menyimpan ke CSV: {e}")
else:
    print("Tidak ada data untuk disimpan.")

Memulai proses scraping...
Mengambil data dari halaman pencarian: https://www.detik.com/search/searchall?query=blt&page=1&result_type=relevansi
Ditemukan 12 item berita di halaman pencarian ini.
    Mengambil konten dari: https://news.detik.com/berita/d-7924631/wamendes-80-ribu-kopdes-merah-putih-launching-juli-bisa-salurkan-blt-bansos
      Peringatan: 'div.media_desc' TIDAK ditemukan di https://news.detik.com/berita/d-7924631/wamendes-80-ribu-kopdes-merah-putih-launching-juli-bisa-salurkan-blt-bansos. Mencoba fallback (area konten umum + <p>).
      Konten ditemukan via FALLBACK (area konten umum + <p>). Panjang: 3187 chars.
  -> Data artikel ke-1 BERHASIL diproses: Wamendes: 80 Ribu Kopdes Merah Putih Launching Juli, Bisa Salurkan BLT-Bansos (Panjang Konten: 3187 chars)
    Mengambil konten dari: https://finance.detik.com/energi/d-7622373/subsidi-bbm-listrik-diganti-ke-blt-tergantung-keputusan-prabowo
      Peringatan: 'div.media_desc' TIDAK ditemukan di https://finance.detik.com/en